In [50]:
import pyterrier as pt
from pyterrier.measures import *
if not pt.started():
  pt.init()

In [27]:
index_path = "E:/index"
indexref = pt.IndexRef.of(index_path)

In [28]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3213835
Number of terms: 16168096
Number of postings: 905088837
Number of fields: 0
Number of tokens: 2204592607
Field names: []
Positions:   true



In [112]:
topics_path = "E:/topics.msmarco-doc.test.txt"
qrels_path = "E:/2019qrels-docs.txt"

qrels = pt.io.read_qrels(qrels_path)
topics = pt.io.read_topics(topics_path,format="singleline")

13:24:44.894 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [113]:
BM25_br = pt.BatchRetrieve(index, wmodel="BM25")
res = BM25_br.transform(topics)

In [114]:
print(res.sort_values(by='score'))

             qid    docid     docno  rank       score  \
630043   1124335  1898702   D330043   999   -0.015792   
630042   1124335  1204457  D3165598   998   -0.015748   
630041   1124335   205442  D1222024   997   -0.015714   
630039   1124335   176508   D255921   995   -0.015710   
630040   1124335  2326967  D3218250   996   -0.015710   
...          ...      ...       ...   ...         ...   
14002     975821  3201650     D1113     2  101.707623   
14001     975821  1254670     D1112     1  101.996935   
5092429   992595  1273431  D3549096     0  103.486367   
14000     975821  1115079     D1115     0  105.921686   
2501786   455425   831991  D3380531     0  133.053255   

                                                     query  
630043                                    time in kamloops  
630042                                    time in kamloops  
630041                                    time in kamloops  
630039                                    time in kamloops  
630040    

In [115]:
pt.Experiment(
    [BM25_br],
    topics,
    qrels,
    eval_metrics=["map", nDCG@10,'mrt','ndcg'],
    )

,name,map,nDCG@10,ndcg,mrt
0,BR(BM25),0.335407,0.538802,0.604765,112.082854


In [116]:
pt.Utils.evaluate(res, qrels, metrics = [nDCG@10,'map'],perquery=True)

defaultdict(dict,
            {'156493': {'map': 0.7390316099528894,
              'nDCG@10': 0.677326101187837},
             '1110199': {'map': 0.17178550167029247,
              'nDCG@10': 0.26998539222812484},
             '1063750': {'map': 0.029627818536490165,
              'nDCG@10': 0.18115419260123777},
             '130510': {'map': 0.5156064218649112,
              'nDCG@10': 0.8857316381822387},
             '489204': {'map': 0.040938477378802,
              'nDCG@10': 0.18760137034475094},
             '573724': {'map': 0.3763646711777269,
              'nDCG@10': 0.6927298377998571},
             '1133167': {'map': 0.5714597025081728,
              'nDCG@10': 0.7196073311817376},
             '527433': {'map': 0.20261189166853513,
              'nDCG@10': 0.29735787004447234},
             '1037798': {'map': 0.011802883314292094,
              'nDCG@10': 0.1233781076433543},
             '915593': {'map': 0.41131644828748226,
              'nDCG@10': 0.2797914894814979},

In [81]:
print(BM25_br.search("what is theraderm used for")[:10])#855410
print(BM25_br.search("definition declaratory judgment")[:10]) #130510
print(BM25_br.search("how long to hold bow in yoga")[:10])#1132213

print(BM25_br.search("who is robert gray")[:10])#1037798
print(BM25_br.search("lps laws definition")[:10])#443396

  qid    docid     docno  rank      score                       query
0   1   859408  D3494218     0  39.830542  what is theraderm used for
1   1  3067531  D3494217     1  37.038080  what is theraderm used for
2   1  1896050  D2536093     2  34.435514  what is theraderm used for
3   1  1335934  D3494220     3  32.731794  what is theraderm used for
4   1  2844917  D3494221     4  12.810839  what is theraderm used for
5   1   226907  D1448579     5  10.056820  what is theraderm used for
  qid    docid     docno  rank      score                            query
0   1   392325  D3480062     0  42.800585  definition declaratory judgment
1   1   937555  D3480067     1  41.392746  definition declaratory judgment
2   1  2320429  D3480061     2  39.359881  definition declaratory judgment
3   1   422768  D1672849     3  37.536021  definition declaratory judgment
4   1  2319283  D1672850     4  37.113972  definition declaratory judgment
5   1  1865422  D1161895     5  37.068511  definition declar

In [98]:
pt.io.write_results(res,"result1.res")
pt.Utils.evaluate(res, qrels, metrics = [nDCG@10,'map'])

{'nDCG@10': 0.5388020905969735, 'map': 0.3354068744746378}

In [96]:
doi = index.getDocumentIndex()
di = index.getDirectIndex()
lex = index.getLexicon()
for posting in di.getPostings(doi.getDocumentEntry(937555)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print(lee.getKey())

law
find
http
can
order
need
org
question
help
non
know
opinion
part
express
type
chang
share
definit
ad
right
sign
check
act
last
small
back
feder
debt
busi
bankruptci
page
collect
name
practic
simpli
term
tactic
web
blog
parti
declar
mistak
written
releas
court
motion
claim
legal
default
intern
file
summari
topic
malaysia
code
eleven
mortgag
judgment
copi
consent
collector
judgement
violat
judici
statut
divorc
wipe
maiden
thelawdictionari
foreclosur
rem
grandpar
personam
declaratori
incumbranc
dictionaryoutrop
declaratorylink


In [57]:
#New one Theo said
BM25_br_new = pt.BatchRetrieve(index, wmodel="BM25",controls={"qe":"on"})
res_new = BM25_br_new.transform(topics)

19:55:47.097 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1


In [58]:
pt.Utils.evaluate(res_new, qrels, metrics = [nDCG@10,'map'])

{'nDCG@10': 0.5681765252369344, 'map': 0.3849193821948344}

In [22]:
pipeline_fetch = pt.BatchRetrieve(index, wmodel="BM25")
pipeline_features = pt.FeaturesBatchRetrieve(index,features=["WMODEL:Tf", "WMODEL:PL2"])

train_qrels_path = "E:/msmarco-doctrain-top100.txt"
train_qrels = pt.io.read_qrels(train_qrels_path)
validation_qrels_path = "E:/docdev-stopstem.xml_1.out"
validation_qrels = pt.io.read_qrels(validation_qrels_path)

import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10,25],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
print("<-We are here")
lmart_l_pipe = (pipeline_fetch%100) >> pipeline_features >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
print("<-Now there")
lmart_l_pipe.fit(topics, train_qrels, topics, train_qrels)

<-We are here
<-Now there


C:\Users\alber\anaconda3\lib\site-packages\lightgbm\sklearn.py:621: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
C:\Users\alber\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=100, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=100
[1]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[2]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[3]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[4]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[5]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[6]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[7]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[8]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[9]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1
[10]	valid_0's ndcg@10: 1	valid_0's ndcg@25: 1


In [31]:
test_qrels_path = "E:/msmarco-doctest2019-top100.txt"
test_qrels = pt.io.read_qrels(test_qrels_path)
test_topics_path = "E:/topics.msmarco-doc.test.txt"
test_topics = pt.io.read_topics(test_topics_path,format="singleline")
pt.Experiment(
    [lmart_l_pipe],
    test_topics,
    test_qrels,
    ['ndcg','map','recip_rank'],
    filter_by_topics = False,
    filter_by_qrels = False,
    names=["LambdaMART (LightGBM)"]
)

13:39:14.140 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


KeyboardInterrupt: 